Продолжим совершенствовать код для соревнования [Catch Me If You Can](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2).

Как работать с малыми данными, мы разбирали на семинаре. Но что делать, когда данных очень много, как у нас в части признаков про посещенные сайты?
Мы можем, например, постараться уменьшить размерность признаков и обучить модели на урезанных представлениях данных.

В этом блокноте разберем два метода TruncatedSVD и MiniBatchKMeans, предназанченные для работы с большими данными и вы увидите, как комфортно ими пользоваться. Для контраста можете запустить тот же KMeans и проверить время работы обоих алгоритмов.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import save_npz, load_npz
from sklearn.metrics import silhouette_score

%matplotlib inline

In [ ]:
# Когда вы создали site1 и ttest в прошлом ноутбуке,
# То вы можете сохранить матрицы, чтобы каждый раз их не пересчитывать:
# save_npz('data.npz', site1)
# save_npz('test.npz', ttest)

# Загрузка осуществляется с помощью метода:
site1 = load_npz('data.npz')
ttest = load_npz('test.npz')

In [ ]:
from sklearn.decomposition import TruncatedSVD
# Этот метод является PCA для sparse данных
# Из доков: Contrary to PCA, this estimator does not center the data 
# before computing the singular value decomposition. 
# This means it can work with scipy.sparse matrices efficiently.

In [ ]:
svd = TruncatedSVD(n_components=100, random_state=42)
svd.fit(site1) #обучаем модель

In [ ]:
print(svd.explained_variance_ratio_.sum())
# Попробуйте 500 или 1000 n_components у алгоритма, чтобы сохранить до 90% дисперсии
# В конце ноутбука приведен скор на лидерборде при 100 компонентах и при 1000.
# Спойлер - при тысяче лучше.

In [ ]:
svd_train = svd.transform(site1) #применяем изменение
svd_test = svd.transform(ttest) #применяем изменение

In [ ]:
data = pd.read_csv('../data/Alice/train_sessions.csv')

In [ ]:
target = data['target']

In [ ]:
from sklearn.cluster import MiniBatchKMeans
# Попробуем специальный метод кластеризации для больших данных
# Можете попробовать и другие, но ожидайте, что работать они будут долго

In [ ]:
Ks = range(1, 25)
score = [MiniBatchKMeans(n_clusters=i, random_state=42).fit(svd_train).inertia_ for i in Ks]

In [ ]:
fig = plt.figure(figsize=(10, 5))
plt.plot(Ks, score)

In [ ]:
clst = MiniBatchKMeans(n_clusters=20, random_state=42)
clst.fit(svd_train)
pred = clst.predict(svd_train)
pred_test = clst.predict(svd_test)

In [ ]:
svd2 = TruncatedSVD(n_components=2, random_state=42)
svd2.fit(svd_train) #обучаем для визуализации

In [ ]:
res = svd2.transform(svd_train)
res_test = svd2.transform(svd_test)

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(res[:,0], res[:,1], c=pred, s=1, cmap='viridis')
plt.title('Kmeans')
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(res[:,0], res[:,1], c=target, s=1, cmap='viridis_r')
plt.title('Real')

In [ ]:
# Для теста

plt.figure(figsize=(12,8))
plt.scatter(res_test[:,0], res_test[:,1], c=pred_test, s=1, cmap='viridis_r')
plt.title('Test')

In [ ]:
# Так сразу и не понять, есть ли что-то полезное в кластеризации.
# Поэтому попробуем взять наши признаки из TruncatedSVD и Kmeans и построить случайный лес.

In [ ]:
train = np.concatenate((svd_train, np.array(pred).reshape(-1, 1)), axis=1) 
test = np.concatenate((svd_test, np.array(pred_test).reshape(-1, 1)), axis=1)
# Объединяем признаки

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score as auc
from sklearn.model_selection import KFold
import time

In [ ]:
#обучаем с кросс-валидацией случайный лес
#предсказываем 10 раз трейн, дальше усредним
answ = []
v_metric = []

n=1

kf = KFold(n_splits=10, shuffle=True, random_state=777)   
for tr_ind, val_ind in kf.split(train):
    print('Start {} fold'.format(n))

    val = train[val_ind]
    ttt = train[tr_ind] 

    start_time = time.time()
    clf = RandomForestClassifier(n_estimators=400, 
                                 min_samples_split=10, 
                                 max_depth=8, 
                                 n_jobs=-1, )

    clf.fit(ttt, data['target'][tr_ind].reset_index(drop=True)) 

    model_pred_valid = clf.predict_proba(val)[:, 1]

    y_valid = data['target'][val_ind].reset_index(drop=True)
    valid_metric = auc(y_valid, model_pred_valid)
    v_metric.append(valid_metric)

    print('fold score:', valid_metric, round((time.time() - start_time)/60, 2))
    model_pred = clf.predict_proba(test)[:, 1]
    answ.append(model_pred)

    n+=1

    print('crossval score:', np.mean(v_metric), 'std', np.std(v_metric))
    print('---------------------------------------')

In [ ]:
#собираем предсказания теста
answ_df = pd.DataFrame()
for i in range(len(answ)):
    answ_df['an'+str(i)] = answ[i]
answ_df['answer'] = answ_df.mean(axis=1)

In [ ]:
answ_df.head()

In [ ]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [ ]:
y_test = np.array(answ_df['answer'])

In [ ]:
write_to_submission_file(y_test, 'submission_tree2.csv')

* 100  components, score: 0.85509
* 1000 components, score: 0.88471

Скорее всего, если по сетке подбирать параметры, то можно еще выше скор получить.


### Задание. Срок 5 декабря.

* Какие методы уменьшения размерности и кластеризации предназначены для работы с большими данными (больше нескольких десятков тысяч строк)?
* Сравните на текущих спарс данных насколько по-разному работают предназначенные и нет для больших данных методы уменьшения размерности и кластеризации.
* Подберите гиперпараметры моделей (того же RandomForest) так, чтобы получить модель лучше по кросс-валидации и скору на лб.
* Почему линейная модель показывает себя лучше (если показывает)?